In [1]:
!pip install pandas pyarrow faiss-cpu tf-keras sentence_transformers huggingface_hub[hf_xet] --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import faiss

2025-05-18 17:44:16.177370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 17:44:16.370453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747590256.445612       4 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747590256.465974       4 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747590256.620438       4 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2') 

/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 34: CUDA driver is a stub library (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
index = faiss.read_index('/opt/data/yugioh.index')

In [5]:
df = pd.read_parquet('/opt/data/yugioh_cards.parquet')

In [6]:
def search(query, k=5):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec).astype('float32'), k)
    return df.iloc[I[0]][['name',]]

In [39]:
query = "Elemental"
types = ["Spell Card"]
embedding = model.encode([query])
mask = df['type'] == "Spell Card" # .isin(types)

In [47]:
filtered_embeddings = np.array(embedding)
D, I = index.search(filtered_embeddings.astype('float32'), 100)
filtered_cards = df.iloc[I[0]].loc[mask]

In [48]:
filtered_cards = df.iloc[I[0]].loc[mask]
results = [{
            "name": filtered_cards.iloc[row]['name'],
            "image_url": filtered_cards.iloc[row]['card_images'].tolist(), # array of dict
            "desc": filtered_cards.iloc[row]['desc'], # list of dict
            "distance": int(D[0][row])} 
        for row in range(filtered_cards.shape[0])]

In [50]:
results

[{'name': 'Neo Space',
  'image_url': [{'id': 42015635,
    'image_url': 'https://images.ygoprodeck.com/images/cards/42015635.jpg',
    'image_url_cropped': 'https://images.ygoprodeck.com/images/cards_cropped/42015635.jpg',
    'image_url_small': 'https://images.ygoprodeck.com/images/cards_small/42015635.jpg'}],
  'desc': '"Elemental HERO Neos" and all Fusion Monsters that specifically list "Elemental HERO Neos" as a Fusion Material gain 500 ATK. Fusion Monsters that specifically list "Elemental HERO Neos" as a Fusion Material do not have to activate their effects during the End Phase that shuffle them into the Extra Deck.',
  'distance': 0},
 {'name': 'Miracle Fusion',
  'image_url': [{'id': 45906428,
    'image_url': 'https://images.ygoprodeck.com/images/cards/45906428.jpg',
    'image_url_cropped': 'https://images.ygoprodeck.com/images/cards_cropped/45906428.jpg',
    'image_url_small': 'https://images.ygoprodeck.com/images/cards_small/45906428.jpg'}],
  'desc': 'Fusion Summon 1 "El